<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/Cross-Domain/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CycleGAN

# CycleGAN 의 원리



CycleGAN 의 목적은 가짜 타깃 데이터 y'을 생성기를 통해 생성하는데에 있다.
가짜 이미지 y' 을 x 의 함수로 생성하기 위해서, 소스도메인의 실제이미지 x 와 타깃 도메인의 실제 이미지 y를 활용함으로써 비지도 방식으로 훈련된다.

일반 GAN 과의 차이점은 CycleGAN 은 순환-일관성(cycle-consistency) 제약을 둔다.
전방 순환의 경우 생성기 G 와 F 에 의해 실제 소스 데이터가 가짜 타깃 데이터로부터 재구성 될 수 있다고 가정하는 것이다. 

이를 수식으로 표현하면 다음과 같다.

* ***X' = F(G(X))***

마찬가지로 후방 순환에 대해서도 타깃 데이터 y 는 네트워크에 의해 재구성 될 수 있다. 

전방 순환과 후방순환에서의 일관성을 바탕으로하여 L1 손실을 최소화 시키는 방향으로 훈련을 해야하며 두 순환의 손실을 더한 값을 순환-일관성 손실이라고 한다.

L1, MAE (Mean Absolute Error, MAE) 를 사용하는 이유는 이것이 L2(MSE) 보다 재구성 된 결과 이미지가 더 선명하게 주어지는 것이 Cycle GAN 의 논문에서 제시되었다. 

다른 GAN 모델과 마찬가지로 CycleGAN의 궁극적 목표는 생성기가 판별기를 속일 수 있는 가짜 타깃 데이터를 합성하는 방법을 학습하는 것이다. 본 예제에서는 LSGAN 에서 품질이 더 낫다는 것을 토대로 이진 교차 엔트로피 손실 대신 MSE 손실을 사용한다.

GAN 의 손실은 순환 일관성 검사에 더 무게를 두어 전체 GAN 의 손실로 정의한다.
Cycle GAN 의 훈련 방법은 GAN 과 비슷하다 대략적인 훈련의 Step 은 다음과 같다

- 1. 실제 소스와 타깃 데이터를 사용해 전방-순환 판별기를 훈련해 전방순환 손실을 최소화 한다. 실제 타깃 데이터 배치인 y의 레이블은 1 이고 가짜 타깃 데이터 배치인 y' 의 레이블은 0 으로 한다.

- 2. 실제 소스와 타깃데이터를 사용해 후방 순환 판별기를 훈련해 후방순환 손실을 최소화 한다.실제 소스 데이터 배치인 x 의 레이블은 1이고 가짜 소스 데이터인 x' 의 레이블은 0 으로 학습한다ㅏ.

- 3. 적대적 신경망에서 전방 순환 생성기와 후방 순환 생성기를 훈련해 GAN 의 손실과 순환 일관성의 손실을 최소화 한다. 이때 가짜 타깃 배치와 가짜 소스 배치의 레이블을 1로 하고 판별기의 가중치를 고정한다.

# CIFAR 10 의 Coloration 예제

본 예제에서는 기존 AutoEncoder 의 Coloration 을 CycleGAN 으로 재현한다.
생성기는 U-Net 을 판별기는 Patch GAN 을 사용하였다.

CIFAR10 세트가 32x32 의 사이즈 이기에 더 큰 크기의 이미지를 사용할 경우 U-Net의 입력/출력 차원이 높아질 경우 인코더/디코더의 깊이가 깊어져야하며, 이것이 문제가 될 수 있다. 본래 논문에서는 256 x 256 사이즈의 이미지를 사용하며, ResNet 의 경우도 소개되어 있으니 참고하도록 하자.

인코더 계층은 IN-LeakyReLU-Conv2D 로 구성되며 디코더 계층은 IN-ReLU-Conv2DTranspose 로 구성된다.

스타일 변경에서는 Instance Normalization 을 사용하는데 데이터 샘플마다 적용되는 Batch Normalization 의 역할을 한다. 즉 IN 은 이미지 또는 특징마다 수행되는 BN 이라고 생각할 수 있는데, 이는 스타일 전이에서는 배치가 아니라 샘플마다 contrast를 정규화 하는 것이 더 중요하기 때문이다.

이미지가 클 경우, 한자릿수의 실제 혹은 가짜 확률로 이미지를 계산하는 것은 매개변수로 비효율 적이며, 생성기에서의 이미지 품질을 떨어뜨리는 결과를 가져온다. 이문제에 대한 해결책으로 위에서와 같이 PatchGAN 을 사용하는데 기존의 출력을 2X2 의 Patch 로써 4개가 되는 것이다.

이미지의 크기가 커질수록 Patch 가 많을 경우 이미지가 더 실제와 같이 변환된다.
본 예제에서는 32X32 사이즈이기에 2x2 출력을 사용한다.

In [ ]:
import numpy as np
import os
import sys
from tensorflow.keras.layers import Input, Dropout, Concatenate, add
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LeakyReLU, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2

# 모델 함수 정의


In [ ]:
#모델 구성 단위 블럭 정의
def encoder_layer(inputs, filters=16, kernel_size=3, strides=2, activation='relu', instance_norm=True):
  # Conv2D - IN - LeakyReLU 로 구성된 일반 인코더 계층을 구성한다.
  conv = Conv2D(filters=filters,kernel_size=kernel_size, strides=strides, padding='same')
  x = inputs
  if instance_norm:
    x = InstanceNormalization()(x)
  if activation == 'relu':
    x = Activation('relu')(x)
  else:
    x = LeakyReLU(alpha=0.2)(x)
  
  x = conv(x)
  return x

def decoder_layer(inputs, paired_inputs, filters=16, kernel_size=3, strides=2,activation='relu', instance_norm=True):
  #Conv2DTranspose-IN-LeakyReLU로 구성된 디코더 계층구성, 활성화 함수는 ReLU 로 교체될 수 있음
  #paired_inputs 는 U-net 의 skip connection 을 의미하며 입력에 연결된다.

  conv=Conv2DTranspose(filters=filters,kernel_size=kernel_size, strides=strides, padding='same')
  
  x = inputs
  if instance_norm:
    x = InstanceNormalization()(x)
  if activation=='relu':
    x = Activation('relu')(x)
  else:
    x = LeakyReLU(alpha=0.2)(x)
  
  x = conv(x)
  x = concatenate([x, paired_inputs])
  return x

In [ ]:
#생성기 U-Net Build
def build_generator(input_shape, output_shape=None, kernel_size=3, name=None):
  # 4계층 인코더와 4계층 디코더로 구성된 U-Net 을 구성한다.

  inputs = Input(shape=input_shape)
  channels = int(output_shape[-1])

  e1 = encoder_layer(inputs, filters=32, kernel_size=kernel_size, activation='leaky_relu', stride=1)
  e2 = encoder_layer(e1, filters=64, kernel_size= kernel_size, activation='leaky_relu')
  e3 = encoder_layer(e2, filters=128, kernel_size= kernel_size, activation='leaky_relu')
  e4 = encoder_layer(e3, filters=128, kernel_size= kernel_size, activation='leaky_relu')

  d1 = decoder_layer(e4,e3,filters=128,kernel_size=kernel_size)
  d2 = decoder_layer(d1,e2,filters=64,kernel_size=kernel_size)
  d3 = decoder_layer(d2,e1,filters=64,kernel_size=kernel_size)

  ourtputs = Conv2DTranspoze(channels, kernel_size=kernel_size,stride=1, activation='sigmoid',padding='same')(d3)

  generator = Model(inputs, outputs, name=name)
  
  return generator

#PatchGAN Discriminator Build
def build_discriminator(input_shape,kernel_size=3, patchgan=True,name=None):
  inputs = Input(shape=input_shape)
  x = encoder_layer(inputs,32,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 64,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 128,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 256,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)

  #patchgan=True 이면 n x n 차원 확률 출력 사용
  if patchgan:
    x = LeakyReLU(alpha=0.2)(x)
    outputs = Conv2D(1, kernel_size=kernel_size, strides=1, padding='same')(x)
  else:
    x = Flatten()(x)
    x = Dense(1)(x)
    outputs - Activation('linear')(x)

  discriminator = Model(inputs, outputs, name=name)

  return discriminator

# CycleGAN Build 및 훈련

아래에서 동질성 Loss identify 는 색상을 재현하는데 있어, 발생한 문제에 대해 본래의 색상을 찾아가도록 훈련시키는 과정이다. 이를 위해 타겟데이터 y 가 들어왔을 때 본래 이미지로 재구성할 수 있는 능력을 훈련시키는 것이다.

In [ ]:
def build_cyclegan(shape, source_name='source', target_name='target', kernel_size=3,patchgan=False, identify=False):
  #Cycle GAN 의 구성
  """
  1) 타깃과 소스의 판별기
  2) 타깃과 소스의 생성기
  3) 적대적 네트워크 구성
  """

  #입력 인수
  """
  shape(tuple): 소스와 타깃 형상
  source_name (string): 판별기/생성기 모델 이름 뒤에 붙는 소스이름 문자열
  target_name (string): 판별기/생성기 모델 이름 뒤에 붙는 타깃이름 문자열
  target_size (int): 인코더/디코더 혹은 판별기/생성기 모델에 사용될 커널 크기
  patchgan(bool): 판별기에 patchgan 사용 여부
  identify(bool): 동질성 사용 여부
  """
  #출력 결과:
  #list : 2개의 생성기, 2개의 판별기, 1개의 적대적 모델 

  source_shape, target_shape = shapes
  lr = 2e-4
  decay = 6e-8
  gt_name = "gen_" + target_name
  gs_name = "gen_" + source_name
  dt_name = "dis_" + target_name
  ds_name = "dis_" + source_name

  #타깃과 소스 생성기 구성
  g_target = build_generater(source_shape, target_shape, kernel_size=kernel_size, name=gt_name)
  g_source = build_generater(target_shape, source_shape, kernel_size=kernel_size, name=gs_name)

  print('-----Target Generator-----')
  g_target.summary()
  print('-----Source Generator-----')
  g_source.summary()

  #타깃과 소스 판별기 구성
  d_target = build_discriminator(target_shape, patchgan=patchgan, kernel_size=kernel_size,name=dt_name)
  d_source = build_discriminator(source_shape, patchgan=patchgan, kernel_size=kernel_size,name=ds_name)

  print('-----Targent Generator-----')
  d_target.summary()
  print('-----Source Generator-----')
  d_source.summary()

  optimizer = RMSprop(lr=lr, decay=decay)
  d_target.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
  d_source.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

  #적대적 모델에서 판별기 가중치 고정
  d_target.trainable=False
  d_source.trainable=False

  #적대적 모델의 계산 그래프 구성

  #전방순환 네트워크와 타깃 판별기 
  source_input = Input(shape=source_shape)
  fake_target = g_target(source_input)
  preal_target = d_target(fake_target)
  reco_source = g_source(fake_target)

  #후방순환 네트워크와 타깃 판별기
  target_input = Input(shape=target_shape)
  fake_source = g_source(target_input)
  preal_source = g_source(fake_source)
  reco_target = g_target(fake_source)

  #동질성 손실 사용 시, 두개의 손실항과 출력을 추가한다.
  if identify:
    iden_source = g_source(source_input)
    iden_target = g_target(target_input)
    loss = ['mse', 'mse', 'mae', 'mae', 'mae', 'mae']
    loss_weight = [1.,1.,10.,10.,0.5,0.5]
    inputs = [source_input, target_input]
    outputs = [preal_source, preal_target, reco_source, reco_target, iden_source, iden_target]

  else:
    loss = ['mse', 'mse', 'mae', 'mae']
    loss_weights = [1., 1., 10., 10.]
    inputs = [source_input, target_input]
    outputs = [preal_source, preal_target, reco_source, reco_target]

  #적대적 모델 구성
  adv = Model(inputs, outputs, name='adversarial')
  optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
  adv.compile(loss=loss, loss_weight=loss_weight, optimizer=optimizer,metrics=['accuracy'])
  print('-----Adversarial Network-----')
  adv.summary()

  return g_source, g_target, d_source, d_target, adv

SyntaxError: ignored

In [ ]:
ㅊ